In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

from selenium.webdriver.chrome.service import Service

s = Service("C:/path/to/chromedriver.exe")
driver = webdriver.Chrome(service=s)


url = 'https://sena.territorio.la/index.php?login=true'
path = 'chromedriver.exe'

driver = webdriver.Chrome(path)
driver.get(url)

user='1095910391'
passwd='1095910391'

input_user=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div/form/table/tbody/tr[5]/td/input')
input_passwd=driver.find_element(By.XPATH,'/html/body/div[3]/div/div/div/form/table/tbody/tr[7]/td/label/input')

input_user.send_keys(user)
input_passwd.send_keys(passwd)

boton = driver.find_element(By.XPATH,'//*[@id="ingresar"]')
boton.click()

time.sleep(2)
course = driver.find_element(By.XPATH,'/html/body/div[12]/div/div[2]/div[1]/div[3]/div[2]/ul/li[1]/div/div[2]/span[1]/div/a')
course.click()

time.sleep(2)
evidencia = driver.find_element(By.XPATH,'/html/body/div[15]/table[1]/tbody/tr/td[1]/div/ul/li/ul/li[7]/a')
evidencia.click()
resultados = {}

for i in range(1, 165):  # Iterar N veces
    time.sleep(4)
    titulo = driver.find_element(By.XPATH, f'/html/body/div[15]/table[1]/tbody/tr/td[2]/div[3]/div[3]/div[1]/div[4]/div[{i}]/table/tbody/tr/td[1]/a').text
    calificacion = driver.find_element(By.XPATH, f'/html/body/div[15]/table[1]/tbody/tr/td[2]/div[3]/div[3]/div[1]/div[4]/div[{i}]/table/tbody/tr/td[4]/span').text
    resultados[titulo] = calificacion

print(resultados)

In [41]:
#postgressql
import psycopg2

# Conectarse a la base de datos
conn = psycopg2.connect(
    host="127.0.0.1",
    database="pcl",
    user="postgres",
    password="123456"
)
# Abrir un cursor
cur = conn.cursor()

# Iterar sobre el diccionario y guardar los datos en la base de datos
for titulo, calificacion in resultados.items():
    query = f"INSERT INTO adso.scraping_web (titulo, calificacion) VALUES ('{titulo}', '{calificacion}')"
    cur.execute(query)

# Guardar los cambios y cerrar la conexión
conn.commit()
cur.close()
conn.close()


In [ ]:
#Mysql
import mysql.connector

# Conectarse a la base de datos
cnx = mysql.connector.connect(
    host="tu_host",
    user="tu_usuario",
    password="tu_contraseña",
    database="tu_basedatos"
)

# Abrir un cursor
cursor = cnx.cursor()

# Iterar sobre el diccionario y guardar los datos en la base de datos
for titulo, calificacion in resultados.items():
    query = "INSERT INTO nombre_tabla (titulo, calificacion) VALUES (%s, %s)"
    data = (titulo, calificacion)
    cursor.execute(query, data)

# Guardar los cambios y cerrar la conexión
cnx.commit()
cursor.close()
cnx.close()
